# Task 4 (Guided Exercise)

This notebook is part of Milestone 3, Question 4, and is a guided exercise. I have included guidelines and helpful links (as comments) along with this notebook to guide you through the exercise. Daniel's tutorial will also be helpful, but even if you haven't completed it, you should be able to finish this exercise with the help of the links and guidelines provided here. 

For some of you, this may be the first time exploring a package independently using documentation, so it might take some time to get used to it. But with practice, you will get better at it. At work, you may come across many packages that you have never used before, and you will have to learn them on your own. So, this is a good practice to get used to it.

In this exercise, you will use Spark's MLlib. The idea is to tune some hyperparameters of a Random Forest to find the optimum model. Once you know the optimum settings, you will train a Random Forest in sklearn (`Milestone3-task3.ipynb` part 2) and save it with joblib (`Milestone3-task3.ipynb` part 2) so that you can use it next week to deploy.

Here, consider MLlib as another Python package that you are using, like scikit-learn. You will see many scikit-learn similar classes and methods available in MLlib for various ML-related tasks. You may also notice that some of them are not yet implemented in MLlib. What you write using the PySpark package will use the spark engine to run your code, and hence, all the benefits of distributed computing that we discussed in class.

Note: Whenever you use Spark, make sure that you refer to the right documentation based on the version you are using. You can select the version of Spark from [here](https://spark.apache.org/docs/)  and go to the correct documentation. In our case, we are using Spark 3.3.1, and here is the link to the Spark documentation that you can refer to:

- [MLlib Documentation](https://spark.apache.org/docs/3.3.1/ml-guide.html)
- [MLlib API Reference](https://spark.apache.org/docs/3.3.1/api/python/reference/pyspark.ml.html)

You may notice that there are RDD-based API and DataFrame-based (Main Guide) API available in the documentation. You want to focus on DataFrame-based API as no one uses RDD-based API these days. We will discuss the difference in class.

Before you start this notebook, make sure that you setup your EMR notebooks, uploaded this notebook there, and the kernel you selected is PySpark.

## Install packages

You only want to install following packages for this exercise:

In [1]:
sc.install_pypi_package("pandas")
sc.install_pypi_package("s3fs")
sc.install_pypi_package("scikit-learn")
sc.install_pypi_package("sklearn")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1681799729363_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…




  Created wheel for sklearn: filename=sklearn-0.0.post4-py3-none-any.whl size=4291 sha256=3b685926ea299494826f9dcbc392d0a5560aeffb6f6605df82949f8f0ec14723
  Stored in directory: /mnt/tmp/pip-ephem-wheel-cache-asv92jyh/wheels/89/65/a1/b406f8c3f0a3c5fe55c18adf9f8e06b38a30f127b086b867d4
Successfully built sklearn





## Import necessary libraries

In [2]:
from pyspark.ml import Pipeline
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import VectorAssembler, UnivariateFeatureSelector
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor as sparkRFR
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from sklearn.model_selection import train_test_split
import pandas as pd

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read the data

To start with; read 100 data points for development purpose. Once your code is ready you should try on the whole dataset.

In [3]:
## Remember by default it looks for credentials in home directory. 
## Makes sure your updated credentials are in home directory
## Where this notebook is running? Clue: It is running on our master node. So you want to ssh into master node and update credentials there.
## or pass credentials explicitly and pass as storage_options=aws_credentials (not a good idea)
# aws_credentials = {"key": "","secret": "","token":""}
# replace with s3 path to your data
## here 100 data points for testing the code, 
# pandas_df = pd.read_csv("s3://mds-s3-9-nikita/output/ml_data_SYD.csv", index_col=0, parse_dates=True).iloc[:100].dropna()
pandas_df = pd.read_csv("s3://mds-s3-9-nikita/output/ml_data_SYD.csv", index_col=0, parse_dates=True).dropna()
feature_cols = list(pandas_df.drop(columns="Observed").columns)
train_data, test_data = train_test_split(pandas_df, test_size=0.2, random_state=123)
train_data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

            ACCESS-CM2  ACCESS-ESM1-5  ...   SAM0-UNICON   TaiESM1
time                                   ...                        
1953-10-26   17.906051       0.837579  ...  6.594400e-01  0.129196
1921-10-22    0.515505       1.911354  ...  2.067648e-01  2.018346
1925-01-22    0.161412       2.666091  ...  1.733777e+00  0.932259
1902-11-21    3.651607       3.117433  ...  1.298833e+01  0.005468
1925-02-17    0.635625      39.042773  ...  2.987665e-01  2.923645
...                ...            ...  ...           ...       ...
1910-04-09    0.148872       0.002789  ...  2.572049e+00  0.108625
1931-02-17    2.564109       0.000679  ...  2.685817e+00  2.015153
1937-07-30    0.112727       0.247349  ...  3.227688e-08  0.972568
1965-10-19    0.264141       0.000000  ...  2.926012e-02  1.126471
1932-01-31    0.061617       0.007578  ...  5.271084e+00  0.303226

[36791 rows x 26 columns]

## Preparing dataset for ML

In [4]:
# Load dataframe and coerce features into a single column called "Features"
# This is a requirement of MLlib
# Here we are converting your pandas dataframe to a spark dataframe, 
# Here "spark" is a spark session I will discuss in class. 
# It is automatically created for you in this notebook.
# read more  here https://blog.knoldus.com/spark-createdataframe-vs-todf/
training = spark.createDataFrame(train_data)
assembler = VectorAssembler(inputCols=feature_cols, outputCol="Features")
training = assembler.transform(training).select("Features", "Observed")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Find best hyperparameter settings

Official Documentation of MLlib, Random forest regression [here](https://spark.apache.org/docs/3.3.1/ml-classification-regression.html#random-forest-regression).

Here we will be mainly using following classes and methods;

- [RandomForestRegressor](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.RandomForestRegressor.html)
- [ParamGridBuilder](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.tuning.ParamGridBuilder.html)
    - addGrid
    - build
- [CrossValidator](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.tuning.CrossValidator.html)
    - fit

Use these parameters for coming up with ideal parameters, you could try more parameters, but make sure you have enough power to do it. But you are required to try only following parameters. This will take around 15 min on entire dataset....

    - Use numTrees as [10, 50,100]
    - maxDepth as [5, 10]
    - bootstrap as [False, True]
    - In the CrossValidator use evaluator to be `RegressionEvaluator(labelCol="Observed")`
    
***Additional reference:*** You can refer to [here](https://www.sparkitecture.io/machine-learning/regression/random-forest) and [here](https://www.silect.is/blog/random-forest-models-in-spark-ml/).

Some additional reading [here](https://projector-video-pdf-converter.datacamp.com/14989/chapter4.pdf)

In [5]:
##Once you finish testing the model on 100 data points, then load entire dataset and run , this could take ~15 min.
## write code below.

# RandomForest model
rf = sparkRFR(featuresCol="Features", labelCol="Observed")

# Parameter grid
grid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 50, 100]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .addGrid(rf.bootstrap, [False, True]) \
    .build() 

# Evaluator
evaluator_reg = RegressionEvaluator(labelCol="Observed")

# Cross validation
cv = CrossValidator(estimator=rf, estimatorParamMaps=grid, evaluator=evaluator_reg)
cvModel = cv.fit(training)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-5:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [6]:
# Print run info
# cvModel is a variable that stores the best model obtained after performing cross-validation (crossval.fit(training))
print("\nBest model")
print("==========")
print(f"\nCV Score: {min(cvModel.avgMetrics):.2f}")
print(f"n_estimators: {cvModel.bestModel.getNumTrees}")
print(f"max_depth: {cvModel.bestModel.getMaxDepth()}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Best model

CV Score: 8.07
n_estimators: 100
max_depth: 5

Exception in thread cell_monitor-6:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'

